<a href="https://colab.research.google.com/github/HaithemH/videos_background_substitution/blob/master/Video_Background_Substitution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run with Colab

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#clone the repo 
!git clone https://github.com/HaithemH/videos_background_substitution
%cd videos_background_substitution
!ls -a         

Cloning into 'videos_background_substitution'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 74 (delta 30), reused 27 (delta 1), pack-reused 0
Unpacking objects: 100% (74/74), done.
/content/videos_background_substitution
.   _assets	Dockerfile  inputs   models   README.md		scripts
..  background	.git	    LICENSE  outputs  requirements.txt	test


In [3]:
#Install dependencies
!pip install -r requirements.txt

     |████████████████████████████████| 69 kB 3.2 MB/s 
  Created wheel for imantics: filename=imantics-0.1.12-py3-none-any.whl size=16031 sha256=48502b6d2fe935d01622ec8a7339433fa1521ba2ab2279ab6ad3b27475319cc7
  Stored in directory: /root/.cache/pip/wheels/da/7c/3e/296fe3ed4eb3bd713e91dee0d0549f12f316d49939a64bdc96
Successfully built imantics


In [10]:
#Run background extraction
#Note: modify vodeo paths in config.json, the algorithm "bg_algo" can be median or gmm
!python ./scripts/bgsubtractor.py

In [11]:
#Run moving objects extraction
#Note: modify vodeo paths in config.json, the algorithm "mo_fg_algo" can be mog or knn
!python ./scripts/mo_subtractor.py

In [ ]:
#Replace background with GrauCat
#Note: modify vodeo paths in config.json, grubcut_iter:"number of iterations" "grubcut_ROI":region of interest
!python ./scripts/grubcut_combiner.py

In [ ]:
#Replace background with DeeplabPascalvoc
#download the model
import requests 
file_url = "https://github.com/ayoolaolafenwa/PixelLib/releases/download/1.1/xception_pascalvoc.pb"
	
r = requests.get(file_url, stream = True) 

with open("/content/videos_background_substitution/models/xception_pascalvoc.pb", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 
#Run inference for background replacement   
!python ./scripts/pascal_voc_combiner.py

# Docker Container 

In [1]:
# Install Docker.
!apt-get -qq install docker.io

Preconfiguring packages ...
Selecting previously unselected package libnfnetlink0:amd64.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../00-libnfnetlink0_1.0.1-3_amd64.deb ...
Unpacking libnfnetlink0:amd64 (1.0.1-3) ...
Selecting previously unselected package pigz.
Preparing to unpack .../01-pigz_2.4-1_amd64.deb ...
Unpacking pigz (2.4-1) ...
Selecting previously unselected package libmnl0:amd64.
Preparing to unpack .../02-libmnl0_1.0.4-2_amd64.deb ...
Unpacking libmnl0:amd64 (1.0.4-2) ...
Selecting previously unselected package iproute2.
Preparing to unpack .../03-iproute2_4.15.0-2ubuntu1.3_amd64.deb ...
Unpacking iproute2 (4.15.0-2ubuntu1.3) ...
Selecting previously unselected package libatm1:amd64.
Preparing to unpack .../04-libatm1_1%3a2.5.1-2build1_amd64.deb ...
Unpacking libatm1:amd64 (1:2.5.1-2build1) ...
Selecting previously unselected package libxtables12:amd64.
Preparing to unpack .../05-libxtables12_1.6.1-2ubuntu2_amd64.deb ..

In [16]:
# Check version.
!docker --version
!pwd

Docker version 20.10.7, build 20.10.7-0ubuntu1~18.04.1
/content/videos_background_substitution


In [19]:
!umount -vl /sys/fs/cgroup/cpuset

umount: /sys/fs/cgroup/cpuset (cgroup) unmounted


In [22]:
%shell
!mount -vt sysfs sysfs /sys -o rw,remount
!mount -vt tmpfs tmpfs /sys/fs/cgroup -o rw,remount
!mount -vt cgroup cgroup /sys/fs/cgroup/cpuset -o rw,remount,cpuset

mount: sysfs mounted on /sys.
mount: tmpfs mounted on /sys/fs/cgroup.
mount: /sys/fs/cgroup/cpuset: mount point not mounted or bad option.


In [ ]:
#Build image
%%shell
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker info
docker network ls
docker build -t bgeditor .
docker images
docker run hello-world
kill $(jobs -p)

In [ ]:
#run image
!docker run -it bgeditor